In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

from xgboost import XGBClassifier
from pmdarima import auto_arima

In [ ]:
X_train = pd.read_parquet("data/X_train.parquet")
X_train_features = pd.read_parquet("data/X_train_features.parquet")
y_train = pd.read_parquet("data/y_train.parquet")

# X_test = pd.read_parquet("X_test.reduced.parquet")
# y_test = pd.read_parquet("y_test.reduced.parquet")

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from scipy.stats import chi2

def piecewise_ar_break_test(series, break_point, segment_length_1=None, segment_length_2=None, max_lag=10):
    """
    Detect structural break at break_point using piecewise AR modeling.
    
    Parameters:
    - series: 1D numpy array or list of time series values
    - break_point: index where the series is split
    - max_lag: maximum lag to consider for AR models
    
    Returns:
    - dict with AICs, log-likelihoods, and p-value of likelihood ratio test
    """
    series = np.array(series)
    s1, s2 = series[:break_point], series[break_point:]
    
    if segment_length_1:
        s1 = s1[-segment_length_1:]
    if segment_length_2:
        s2 = s2[:segment_length_2]

#     # Fit AR models
#     model_full = AutoReg(series, lags=max_lag, old_names=False).fit()
#     model_s1 = AutoReg(s1, lags=max_lag, old_names=False).fit()
#     model_s2 = AutoReg(s2, lags=max_lag, old_names=False).fit()
  
    # Fit AUTO-ARIMA models
    model_full = auto_arima(series, start_p=0, start_q=0, max_p=3, max_q=3,
                   m=1, d=None, seasonal=False, stepwise=True, trace=False, 
                   error_action='ignore', suppress_warnings=True)
    model_s1 = auto_arima(s1, start_p=0, start_q=0, max_p=3, max_q=3,
                           m=1, d=None, seasonal=False, stepwise=True, trace=False, 
                           error_action='ignore', suppress_warnings=True)
    model_s2 = auto_arima(s2, start_p=0, start_q=0, max_p=3, max_q=3,
                       m=1, d=None, seasonal=False, stepwise=True, trace=False, 
                       error_action='ignore', suppress_warnings=True)

    # Compare log-likelihoods
    ll_full = model_full.arima_res_.llf
    ll_piecewise = model_s1.arima_res_.llf + model_s2.arima_res_.llf

    # Likelihood ratio test
    lr_stat = 2 * (ll_piecewise - ll_full)
    df = model_s1.arima_res_.df_model + model_s2.arima_res_.df_model - model_full.arima_res_.df_model
    p_value = chi2.sf(lr_stat, df)

    return {
        'AIC_full': model_full.arima_res_.aic,
        'AIC_piecewise': model_s1.arima_res_.aic + model_s2.arima_res_.aic,
        'LL_full': ll_full,
        'LL_piecewise': ll_piecewise,
        'LR_stat': lr_stat,
        'df': df,
        'p_value': p_value
    }

In [ ]:
# Simulate time series with a break
np.random.seed(42)
predictions = []
for i in tqdm.tqdm(range(0, 10001)):
    series = X_train.loc[i].reset_index()
    break_point = series.groupby('period')['time'].first()[1]

    # Run break test
    result = piecewise_ar_break_test(series['value'], break_point=break_point, segment_length_1=None, segment_length_2=None, max_lag=3)
    predictions.append(result)
    
df_pred = pd.DataFrame(predictions)

In [ ]:
# Calculate roc-auc score for predictions
# df_pred['prediction'] = df_pred['p_value'].apply(lambda p: False if p >= 0.05 else True)
df_pred['prediction_1'] = df_pred.apply(lambda x: x['LL_full'] - x['LL_piecewise'], axis=1)

target = y_train['structural_breakpoint'].replace({False: 0, True: 1})
roc_auc_score(target[:507], df_pred['prediction'])

In [ ]:
## RESULTS
# 1. AUTO_REG (Train : 10,000 samples) :- 0.54
# 2. AUTO_ARIMA (Train : 500 samples) :-  0.52

## \#2

In [ ]:
def get_coefficient_match(param):
    conf_int = 0.05
    if param['pval_coef_period_0'] > conf_int:
        param ['coef_period_0'] = np.nan
     
    if param['pval_coef_period_1'] > conf_int:
        param ['coef_period_1'] = np.nan

    if pd.isna(param['coef_period_0']) or pd.isna(param['coef_period_1']):
        return False
    elif (param['coef_period_0'] > param['coef_period_1']) and (param['0.975_period_1'] < param['0.025_period_0']):
        return False
    elif (param['coef_period_0'] > param['coef_period_1']) and (param['0.975_period_1'] >= param['0.025_period_0']):
        return True
    elif (param['coef_period_0'] < param['coef_period_1']) and (param['0.975_period_0'] < param['0.025_period_1']):
        return False
    elif (param['coef_period_0'] < param['coef_period_1']) and (param['0.975_period_0'] >= param['0.025_period_1']):
        return True
    elif param['coef_period_0'] == param['coef_period_1']:
        return True

def fit_and_compare_arima(data, label, id, n_exceptions):
    period_0 = data.loc[data['period'] == 0].reset_index()
    period_1 = data.loc[data['period'] == 1].reset_index()
    
    model_0 = auto_arima(period_0['value'], start_p=0, start_q=0, max_p=3, max_q=3,
                       m=1, d=None, seasonal=False, stepwise=True, trace=False, 
                       error_action='ignore', suppress_warnings=True)
    model_1 = auto_arima(period_1['value'], start_p=0, start_q=0, max_p=3, max_q=3,
                           m=1, d=None, seasonal=False, stepwise=True, trace=False, 
                           error_action='ignore', suppress_warnings=True)
    model_0_params = pd.concat([model_0.arima_res_.params.to_frame().rename({0: 'coef_period_0'}, axis=1),
                            model_0.arima_res_.conf_int().rename({0: '0.025_period_0', 1: '0.975_period_0'}, axis=1),
                            model_0.arima_res_.pvalues.to_frame().rename({0: 'pval_coef_period_0'}, axis=1)], axis=1)
    model_1_params = pd.concat([model_1.arima_res_.params.to_frame().rename({0: 'coef_period_1'}, axis=1),
                            model_1.arima_res_.conf_int().rename({0: '0.025_period_1', 1: '0.975_period_1'}, axis=1),
                            model_1.arima_res_.pvalues.to_frame().rename({0: 'pval_coef_period_1'}, axis=1)], axis=1)
    model_params = pd.concat([model_0_params, model_1_params], axis=1)  
    model_params['coeff_match'] = model_params.apply(lambda row: get_coefficient_match(row), axis=1)
    model_params = model_params.assign(structural_breakpoint = label['structural_breakpoint'], id = id)
    model_params = model_params.reset_index().rename(columns={'index': 'coefficient'})

    threshold = 0.9
    try:
        result = pd.DataFrame({'id': id, 'structural_breakpoint': label['structural_breakpoint'], 'structural_breakpoint_pred': model_params['coeff_match'].value_counts(normalize=True).loc[False]}, index=[0])
        return result, model_params
    except:
        n_exceptions += 1
        result = pd.DataFrame({'id': id, 'structural_breakpoint': label['structural_breakpoint'],  'structural_breakpoint_pred': 0.5}, index=[0])
        return result, model_params

In [ ]:
df_predictions = pd.DataFrame()
df_params = pd.DataFrame()
n_exceptions = 0

for id in tqdm.tqdm(range(10001, 10102)):
    df = test_data.loc[id]
    label = test_labels.loc[id]

    prediction, params = fit_and_compare_arima(df, label, id, n_exceptions)
    df_predictions = pd.concat([df_predictions, prediction], axis=0, ignore_index=True)
    df_params = pd.concat([df_params, params], axis=0, ignore_index=True)

print(n_exceptions)

In [ ]:
df_predictions

In [ ]:
df_params[df_params['id'] == 10002]

In [ ]:
test_data.loc[10002, 'value'].plot()

In [ ]:
# Compute confusion matrix
# cm = confusion_matrix(df_predictions['structural_breakpoint'], df_predictions['structural_breakpoint_pred'])
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels= [False, True])
# disp.plot(cmap="Blues")
# plt.title("Confusion Matrix")
# plt.show()

In [ ]:
# Get the ROC-AUC score from coefficient match based prediction
roc_auc_score(df_predictions['structural_breakpoint'], df_predictions['structural_breakpoint_pred'])

In [ ]:
coefficient_features = ['coef_period_0', '0.025_period_0', '0.975_period_0', 'pval_coef_period_0',
                        'coef_period_1', '0.025_period_1', '0.975_period_1', 'pval_coef_period_1'
                        'coeff_match']

df_params_features = pd.pivot_table(df_params, index=['id'], columns=['coefficient'], values=coefficient_features)
df_params_features.columns = ['_'.join(col).strip() for col in df_params_features.columns.values]

with pd.option_context('display.max_columns', None):
    display(df_params_features)

In [ ]:
# Fit XGBoost model with AUTO-Arima coefficients
xgb_clf = XGBClassifier(n_estimators=500)

# Get the ROC-AUC score from model
auc_scores = cross_val_score(xgb_clf, df_params_features, labels[:1000], cv=5, scoring='roc_auc')
print(f"Individual AUC scores: {auc_scores}")
print(f"Mean AUC score: {np.mean(auc_scores):.4f}")
print(f"Standard deviation of AUC scores: {np.std(auc_scores):.4f}")

In [ ]:
STOP

In [ ]:
# Infer structural break based on comparison of parameters
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    # model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

    yield  # Mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
        # Baseline approach: Compute t-test between values before and after boundary point
        # The negative p-value is used as our score - smaller p-values (larger negative numbers)
        # indicate more evidence against the null hypothesis that distributions are the same,
        # suggesting a structural break
        
        def predict(df_features: pd.DataFrame):
            prediction = fit_and_compare_arima(df_features)
            return prediction
            
        dataset = dataset.reset_index()
        prediction = predict(dataset)
        yield prediction  # Send the prediction for the current dataset

In [ ]:
values = pd.concat([period_0['value'], period_1['value']], axis=0).reset_index()
values['value']

In [ ]:
predictions = pd.concat([model_0.predict_in_sample(), model_1.predict_in_sample()], axis=0).reset_index()
predictions['predicted_mean']

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 6))
values['value'].plot(color='r', ax=ax)
predictions['predicted_mean'].plot(color='b', ax=ax)
plt.axvline(x = 2407, color='r', linestyle='--')
plt.legend()
plt.show()

## #3

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import entropy

import warnings
warnings.filterwarnings('ignore')

def residual_features(time_series_data, n, max_lag=10):
    """
    Extract residual-based features from two time series segments.
    
    Parameters:
    - segment_1: array-like, first part of the time series
    - segment_2: array-like, second part of the time series
    - max_lag: lag order for AR model
    
    Returns:
    - Dictionary of residual-based features
    """
    features = {}
    
    segment_1 = time_series_data[time_series_data['period'] == 0]['value']
    segment_2 = time_series_data[time_series_data['period'] == 1]['value']


    # Fit AR models
    model_1 = AutoReg(segment_1, lags=max_lag, old_names=False).fit()
    model_2 = AutoReg(segment_2, lags=max_lag, old_names=False).fit()

    resid_1 = model_1.resid
    resid_2 = model_2.resid

    # Residual statistics
    features[f'resid_mean_1_{n}'] = np.mean(resid_1)
    features[f'resid_mean_2_{n}'] = np.mean(resid_2)
    features[f'resid_var_1_{n}'] = np.var(resid_1)
    features[f'resid_var_2_{n}'] = np.var(resid_2)
    features[f'resid_var_ratio_{n}'] = features[f'resid_var_2_{n}'] / (features[f'resid_var_1_{n}'] + 1e-6)
    features[f'resid_var_diff_{n}'] = features[f'resid_var_2_{n}'] - features[f'resid_var_1_{n}']

    # Autocorrelation of residuals
    features[f'resid_acf_1_lag1_{n}'] = pd.Series(resid_1).autocorr(lag=1)
    features[f'resid_acf_2_lag1_{n}'] = pd.Series(resid_2).autocorr(lag=1)

#     # Ljung–Box test
#     lb_1 = acorr_ljungbox(resid_1, lags=[max_lag], return_df=True)
#     lb_2 = acorr_ljungbox(resid_2, lags=[max_lag], return_df=True)
#     features[f'ljung_box_stat_1_{n}'] = lb_1['lb_stat'].values[0]
#     features[f'ljung_box_stat_2_{n}'] = lb_2['lb_stat'].values[0]
#     features[f'ljung_box_pval_1_{n}'] = lb_1['lb_pvalue'].values[0]
#     features[f'ljung_box_pval_2_{n}'] = lb_2['lb_pvalue'].values[0]

#     # Entropy of residuals
#     hist_1 = np.histogram(resid_1, bins=20, density=True)[0]
#     hist_2 = np.histogram(resid_2, bins=20, density=True)[0]
#     features[f'resid_entropy_1_{n}'] = entropy(hist_1 + 1e-6)
#     features[f'resid_entropy_2_{n}'] = entropy(hist_2 + 1e-6)
#     features[f'resid_entropy_diff_{n}'] = features[f'resid_entropy_2_{n}'] - features[f'resid_entropy_1_{n}']

    return pd.DataFrame(features, index=[0])

In [ ]:
# Find breakpoint time & position difference for each 'id'
df = X_train.reset_index()
break_point_times = df[df['period'] == 1].groupby(['id','period'])[['time']].first()\
                                       .rename({'time': 'breakpoint_time'}, axis=1).reset_index()
df = pd.merge(df, break_point_times[['id', 'breakpoint_time']], on='id', how='inner')
df['position_from_breakpoint_time'] = df['time'] - df['breakpoint_time']
pos = df['position_from_breakpoint_time'].values

In [ ]:
windows = [w for w in range(50, 1001, 50)]

df_resid_features = pd.DataFrame()
for n in tqdm.tqdm(windows):
    features = df[~df['position_from_breakpoint_time'].between(-n, n-1)].groupby('id').apply(lambda x: residual_features(x, n, max_lag=5))
    df_resid_features = pd.concat([df_resid_features, features], axis=1)

In [ ]:
df_resid_features

In [ ]:
best_features = ['value_max_jump_pre_600', 'pval_levene_40', 'value_mean_absolute_change_post_650', 'value_ptp_pre_150', 'value_kurtosis_post_850', 'value_mean_post', 'value_min_post_600', 'value_percentile_25_pre_750', 'value_avg_fft_mag_pre_20', 'pval_ks_2samp_200', 'value_downward_steps_pre_850', 'value_max_post', 'value_kurtosis_pre', 'value_percentile_25_pre_850', 'value_percentile_25_post_450', 'pval_levene_100', 'value_mean_absolute_change_pre_250', 'value_min_post_500', 'value_percentile_25_post_250', 'value_num_turning_points_post_800', 'value_max_post_300', 'value_avg_fft_mag_pre_350', 'value_ptp_post_50', 'value_percentile_25_pre_150', 'value_kurtosis_pre_600', 'value_std_post_60', 'value_sum_fft_mag_post_750', 'value_kurtosis_pre_700', 'value_mean_pre_550', 'value_std_pre_20', 'value_avg_fft_mag_post', 'value_volatility_pre_30', 'pval_ks_2samp_90', 'value_mean_absolute_change_post_50', 'pval_ks_2samp_950', 'value_auto_1_post', 'value_auto_2_post', 'value_slope_post_350', 'value_mean_absolute_change_pre_700', 'value_std_pre_700', 'value_kurtosis_post_600', 'value_min_pre_70', 'value_mean_absolute_change_pre_60', 'value_percentile_25_pre_80', 'value_avg_fft_mag_post_650', 'pval_ks_2samp_30', 'value_mean_pre_450', 'value_max_pre_300', 'value_auto_2_pre_900', 'value_percentile_25_post_90', 'value_percentile_25_pre', 'value_min_pre_950', 'value_mean_absolute_change_post_90', 'value_max_post_40', 'value_num_turning_points_pre_10', 'value_upward_steps_pre_950', 'value_ptp_post_250', 'value_avg_fft_mag_post_500', 'value_ptp_pre_250', 'value_std_post_10', 'value_ptp_post_600', 'value_downward_steps_post_200', 'value_sum_fft_mag_post_400', 'value_downward_steps_post', 'value_mean_post_500', 'value_auto_1_post_450', 'value_max_jump_pre_70', 'value_num_turning_points_pre_200', 'pval_levene_400', 'value_mean_post_650', 'value_percentile_25_post_150', 'value_skew_post_800', 'value_std_post_20', 'value_ptp_post_5', 'diff_max_jump', 'pval_ks_2samp_60', 'value_mean_absolute_change_pre_450', 'value_auto_3_pre_350', 'ratio_mean_absolute_change', 'diff_mean', 'value_ptp_pre_500', 'pval_levene_30', 'value_num_turning_points_post_80', 'value_volatility_post_90', 'value_ptp_pre_700', 'value_upward_steps_post_90', 'value_upward_steps_pre_250', 'value_percentile_25_pre_250', 'value_volatility_post_300', 'value_num_turning_points_post_50', 'value_max_jump_pre_10', 'value_volatility_pre_20', 'value_dominant_freq_ind_post_800', 'value_ptp_pre_550', 'value_upward_steps_pre_70', 'value_std_pre_500', 'ratio_std_1000', 'pval_ks_2samp_150', 'pval_ks_2samp_500', 'value_ptp_pre_950', 'value_percentile_75_post_450', 'value_std_post_80', 'pval_ttest_ind_950', 'value_volatility_pre_700', 'value_ptp_pre_650', 'value_upward_steps_post', 'pval_ks_2samp_700', 'value_upward_steps_pre_60', 'value_auto_1_pre_800', 'ratio_mean_absolute_change_1000', 'value_skew_post_400', 'value_auto_3_post_550', 'value_mean_pre_350', 'value_num_turning_points_pre_250', 'pval_ks_2samp_5', 'pval_levene_80', 'value_auto_1_pre_700', 'value_downward_steps_post_500', 'value_trend_post_350', 'ratio_num_turning_points', 'value_volatility_pre_750', 'value_median_pre_250', 'value_trend_post_80', 'value_skew_post', 'value_max_pre_80', 'value_ptp_post_750', 'value_mean_absolute_change_post_60', 'value_avg_fft_mag_pre', 'value_mean_pre_800', 'value_median_pre_150', 'value_percentile_75_pre_600', 'ratio_median', 'pval_ks_2samp_20', 'value_skew_pre_40', 'diff_percentile_25', 'value_trend_pre_150', 'value_kurtosis_post_150', 'value_auto_2_post_550', 'pval_levene_750', 'value_upward_steps_pre_350', 'value_max_jump_pre', 'value_mean_absolute_change_pre_650', 'value_percentile_25_post_500', 'pval_ks_2samp_550', 'value_auto_2_post_300', 'value_max_jump_post_250', 'diff_num_turning_points_1000', 'value_ptp_pre', 'value_mean_absolute_change_pre_20', 'value_dominant_freq_ind_post_950']
X_tr = X_train_features[best_features].join(df_resid_features.droplevel(1))
X_tr

In [ ]:
model = XGBClassifier(n_estimators=1000, objective='binary:logistic',
                      max_depth=8, learning_rate=0.1, n_jobs=-1, random_state=15)

print('RUNNING CROSS VALIDATION :')
roc_auc_scores = cross_val_score(model, X_tr, y_train.values, cv=5, scoring='roc_auc', verbose=1)
print(roc_auc_scores)
print(roc_auc_scores.mean(), roc_auc_scores.std())

In [ ]:
[0.705855 0.72442352 0.73095425 0.72545912 0.7344343]
0.7242252373443077 0.009884653564360298
